In [148]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.linear_model import LogisticRegression  
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier 
from sklearn.tree import DecisionTreeClassifier  
from sklearn.preprocessing import StandardScaler,LabelEncoder  
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
from sklearn.model_selection import train_test_split,RandomizedSearchCV, GridSearchCV,cross_validate 
from sklearn.metrics import (confusion_matrix, precision_score,recall_score,
                              classification_report,make_scorer,fbeta_score,roc_auc_score) 
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.utils import class_weight
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline
import xgboost
import seaborn as sns
import matplotlib.pyplot as plt
from xgboost import XGBClassifier


In [149]:
import warnings
warnings.filterwarnings("ignore")

In [150]:
train = pd.read_csv(r'/kaggle/input/titanic/train.csv')
test = pd.read_csv(r'/kaggle/input/titanic/test.csv')
train.head()

In [151]:
train.info()

In [152]:
sns.countplot(data=train,x='Survived')

Event rate is 38% with 891 rows and few null values in age, cabin, and embarked

In [153]:
train['Survived'].value_counts(normalize = True)

In [154]:
train.describe()

In [155]:
train.columns

In [156]:
train.isnull().sum()

Cabin has almost 77% of the data is missing and it does not make sense to fill the missing values in this case
Embarked is replace with mode of the category, and Age is replaced with median

In [157]:
train.drop(['Cabin','Ticket'],axis=1,inplace=True)
test.drop(['Cabin','Ticket'],axis=1,inplace=True)

# 1.EDA and data visualization 

In [158]:
sns.catplot(data=train, x="Sex", y="Survived", hue="Embarked", kind="bar")
sns.catplot(data=train, x="Sex", y="Survived", hue="Pclass", kind="bar")

In [159]:
train['Age'].hist()

In [160]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(18,12))

sns.boxplot(data=train,x="Survived", y="Age", hue='Sex',ax=ax[0][0])
sns.boxplot(data=train, x='Survived', y="Fare", hue='Sex',ax = ax[0][1])
sns.boxplot(data=train, x='Survived', y="SibSp", ax = ax[1][0])
sns.boxplot(data=train, x='Survived', y="Parch", ax = ax[1][1])

In [161]:
train['Sex'].value_counts()

In [162]:
g = sns.catplot(x="Fare", y="Survived", row="Pclass",
                kind="box", orient="h", height=1.5, aspect=4,
                data=train)
g.set(xscale="log")

In [163]:
g = sns.catplot(x="Fare", y="Survived", row="Embarked",
                kind="box", orient="h", height=1.5, aspect=4,
                data=train)
g.set(xscale="log")

# 2. Correlation

In [164]:
correlation = train.corr(method='pearson')
columns = correlation.nlargest(20, 'Survived').index
columns

In [165]:
plt.figure(figsize=(12,12)) 
correlation_map = np.corrcoef(train[columns].values.T)
sns.set(font_scale=1.0)
heatmap = sns.heatmap(correlation_map, cbar=True, annot=True, square=True, fmt='.2f', yticklabels=columns.values, xticklabels=columns.values)

plt.show()

# 3. Feature engineering 

In [166]:
train.columns

In [167]:
train['Name'].head()

In [168]:
train['Name'] = train['Name'].str.extract('([A-Za-z]+)\.')
test['Name'] = test['Name'].str.extract('([A-Za-z]+)\.')

In [169]:
train['Name'].value_counts()

In [170]:
def convert_title(title):
    if title in ["Ms", "Mile", "Miss"]:
        return "Miss"
    elif title in ["Mme", "Mrs"]:
        return "Mrs"
    elif title == "Mr":
        return "Mr"
    elif title == "Master":
        return "Master"
    else:
        return "Other"
        
train["Title"] = train["Name"].map(convert_title)
test["Title"] = test["Name"].map(convert_title)

train["Title"].value_counts()

In [171]:
train.drop(['Name'],axis=1,inplace=True)
test.drop(['Name'],axis=1,inplace=True)

**Age**

In [172]:
train['Age'].isnull().sum()

In [173]:
train.groupby(['Title'])['Age'].mean()

In [174]:
train.loc[(train['Age'].isnull()) & (train["Title"]=='Master'), 'Age'] = 5
train.loc[(train['Age'].isnull()) & (train["Title"]=='Miss'), 'Age'] = 22
train.loc[(train['Age'].isnull()) & (train["Title"]=='Mr'), 'Age'] = 32
train.loc[(train['Age'].isnull()) & (train["Title"]=='Mrs'), 'Age'] = 36
train.loc[(train['Age'].isnull()) & (train["Title"]=='Other'), 'Age'] = 44

In [175]:
test.loc[(test['Age'].isnull()) & (test["Title"]=='Master'), 'Age'] = 5
test.loc[(test['Age'].isnull()) & (test["Title"]=='Miss'), 'Age'] = 22
test.loc[(test['Age'].isnull()) & (test["Title"]=='Mr'), 'Age'] = 32
test.loc[(test['Age'].isnull()) & (test["Title"]=='Mrs'), 'Age'] = 36
test.loc[(test['Age'].isnull()) & (test["Title"]=='Other'), 'Age'] = 44

**Fare**

In [176]:
test[test["Fare"].isna()]

In [177]:
train.groupby("Pclass").mean()["Fare"]

In [178]:
test['Fare'].fillna(train.groupby("Pclass").mean()["Fare"][3], inplace=True)

**Embarked**

In [179]:
train.loc[train['Embarked'].isnull(),'Embarked'] = train['Embarked'].mode()[0]
test.loc[test['Embarked'].isnull(),'Embarked'] = train['Embarked'].mode()[0]

**Pclass**

Pclass has negatuve correlation with target so to make it positive correlation change the lables

In [180]:
train["Pclass"].replace([1,2,3], [3,2,1], inplace=True)
test["Pclass"].replace([1,2,3], [3,2,1], inplace=True)

In [181]:
train.columns

In [182]:
train['Sex'] = train['Sex'].replace({'male':0,
                                    'female':1})
test['Sex'] = test['Sex'].replace({'male':0,
                                    'female':1})

In [183]:
def encode_and_bind(original_dataframe,features_to_encode):
    dummies = pd.get_dummies(original_dataframe[[features_to_encode]])
    res = pd.concat([original_dataframe,dummies],axis=1)
    return res

train = encode_and_bind(train,'Title')
test = encode_and_bind(test,'Title')

In [184]:
test = encode_and_bind(test,'Embarked')
train = encode_and_bind(train,'Embarked')

In [185]:
train.columns

In [186]:
train.drop(['Embarked','Title'],axis=1,inplace=True)
test.drop(['Embarked','Title'],axis=1,inplace=True)

In [187]:
X_train = train.drop(['PassengerId','Survived'],axis=1)
y_train = train[['Survived']]
#X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42,stratify=y)
X_test = test.copy()

In [188]:
X_train.columns

In [189]:
std = StandardScaler()
cols = ['Age','Fare']
X_train[cols] = std.fit_transform(X_train[cols])
X_test[cols] = std.fit_transform(X_test[cols])

In [190]:
classifiers = {
    "KNN": KNeighborsClassifier(), 
    "LR": LogisticRegression(max_iter=1000), 
    "DT": DecisionTreeClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "XGB": XGBClassifier(),
    "LGBM": LGBMClassifier()
}

results = dict()
for name, clf in classifiers.items():
    model = clf
    cv_results = cross_validate(
        model, X_train, y_train, cv=5,
        scoring=('roc_auc')
    )

    results[name] = cv_results['test_score'].mean()
    
results

In [193]:
lgbm = LGBMClassifier(random_state=0)
params = {
    "boosting_type": ["gbdt", "dart", "goss"],
    "learning_rate": [0.1, 0.05, 0.01],
    "n_estimators": [10, 50, 100, 300]
}
clf = GridSearchCV(lgbm, params, cv=10)
clf.fit(X_train, y_train)
print("Best hyperparameter:", clf.best_params_)

In [196]:
y_train_hat = clf.predict(X_train)
# y_test_hat = lr.predict(y)

print(clf)
print('Logistic Regression')
print('Train performance')
print('-------------------------------------------------------')
print(classification_report(y_train, y_train_hat))

# print('Test performance')
# print('-------------------------------------------------------')
# print(classification_report(y_test, y_test_hat))

print('Roc_auc score')
print('-------------------------------------------------------')
print(roc_auc_score(y_train, y_train_hat))
print('')

print('Confusion matrix')
print('-------------------------------------------------------')
print(confusion_matrix(y_train, y_train_hat))

In [197]:
#feature importance
feat_imp = pd.DataFrame(clf.best_estimator_.feature_importances_,index=X_train.columns, columns=['Importance'])
feat_imp.sort_values(by='Importance',ascending=False,inplace=True)
feat_imp.plot(kind='bar',figsize=(8,6))

In [204]:
#gain curve
import scikitplot as skplt
predict_prob = clf.predict_proba(X_train)
skplt.metrics.plot_cumulative_gain(y_train,predict_prob)
plt.show()

In [200]:
X_test.columns

In [201]:
X_test = X_test.drop(['PassengerId'],axis=1)
y_pred = clf.predict(X_test)

In [202]:
submission = pd.read_csv(r'/kaggle/input/titanic/gender_submission.csv')
submission['Survived'] = y_pred

In [203]:
submission.to_csv(r'/kaggle/working/titanic_prediction_lgb.csv',index=False)